In [56]:
%pip install transformers
%pip install contractions

You should consider upgrading via the 'c:\Users\65831\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.




     ------------------------------------ 287.5/287.5 KB 572.8 kB/s eta 0:00:00


You should consider upgrading via the 'c:\Users\65831\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [19]:
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

import string
import re
import json

import nltk
nltk.download('omw-1.4')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.sentiment import SentimentIntensityAnalyzer

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\65831\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\65831\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\65831\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [20]:
tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")
model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")

In [85]:
def remove_contractions(text):
    text = text.replace("'s","")
    return text

def remove_punctuation(text):
    text = re.sub(r'[^\w\s]', '', text)
    return text

def lowercase(text):
    return text.lower()

def remove_stopwords(text):
    stopwords = nltk.corpus.stopwords.words('english')
    text = ' '.join([word for word in text.split() if word not in stopwords])
    return text

def remove_special_character(text):
    text = text.replace('\n', ' ') 
    return text

def lemmatize(text):
    lem = WordNetLemmatizer()
    corpus = ' '.join([lem.lemmatize(x, pos = 'v') for x in text.split()])
    return corpus

def preprocess_text(text):
    text = remove_contractions(text)
    text = remove_punctuation(text)
    text = lowercase(text)
    text = remove_stopwords(text)
    text = remove_special_character(text)
    text = lemmatize(text)
    return text


In [64]:
def finbert_chunk_process(df) : 
    df = df.copy()
    print("Processing chunk")
    df['Date'] = pd.to_datetime(df['Date'])
    df['Processed Title'] = df['Title'].apply(lambda x: preprocess_text(x))
    # df['Processed Text'] = df['Text'].apply(lambda x: preprocess_text(x))
   
    # title
    news_title = df['Processed Title'].to_list()
    inputs = tokenizer(news_title, padding = True, truncation = True, return_tensors='pt')
    outputs = model(**inputs)

    predictions_title = torch.nn.functional.softmax(outputs.logits, dim=-1)

    model.config.id2label

    df['Positive_Title'] = predictions_title[:, 0].tolist()
    df['Negative_Title'] = predictions_title[:, 1].tolist()
    df['Neutral_Title'] = predictions_title[:, 2].tolist()

    # # text
    # news_text = df['Processed Text'].to_list()
    # inputs = tokenizer(news_text, padding = True, truncation = True, return_tensors='pt')
    # outputs = model(**inputs)

    # predictions_text = torch.nn.functional.softmax(outputs.logits, dim=-1)

    # model.config.id2label

    # df['Positive_Text'] = predictions_text[:, 0].tolist()
    # df['Negative_Text'] = predictions_text[:, 1].tolist()
    # df['Neutral_Text'] = predictions_text[:, 2].tolist()


    return df

def finbert_process(df, chunk_size=50):
    print("Breaking data into smaller chunks")
    chunks = [df[i:i+chunk_size] for i in range(0, len(df), chunk_size)] 

    lst = []
    for chunk in chunks:
        lst.append(finbert_chunk_process(chunk))
    
    output = pd.concat(lst)
    
    return output
    

In [93]:
def sentiment_aggregator(df, title = True, type = "abs_max"):
    """
    Aggregates sentiments on a per day basis.

    Parameters
    ----------
    df: DataFrame
        Dataset generated after sentiment analysis.
    title: boolean
        To indicate if the news title or news body text is used to generate the aggregated sentiment. 
        Default is True (ie. News title is used for aggregated sentiment)
    type: Str {"mean", "abs_max"}
        To indicate method of calculation.
        "mean": Group by Date and takes mean of "Compound"
        "abs_max": Calculates the absolute max of "Positive" and "Negative" column. Then group by Date and takes mean of this new column

    Returns
    -------
    Output : Series
        Contains aggregated sentiment for each day
    """
    if title:
        positive = "Positive_Title"
        negative = "Negative_Title"
    else:
        positive = "Positive_Text"
        negative = "Negative_Text"

    if type == "mean":
        df["Compound"] = df[positive] - df[negative]
        return df.groupby('Date')['Compound'].aggregate('mean')

    elif type == "abs_max":
        df['Negative'] = -df[negative]
        df['Positive'] = df[positive]
        df['Sentiment'] = df.apply(lambda x: max(x["Negative"], x["Positive"], key=abs), axis=1)

        return df.groupby('Date')['Sentiment'].aggregate('mean')


In [103]:
def clean_all_data():
    df_world= pd.read_csv('../Data/1_Scraped/Unstructured_Data/Global/world_news.csv')
    df_politics= pd.read_csv('../Data/1_Scraped/Unstructured_Data/Global/politics_news.csv')
    df_coronavirus= pd.read_csv('../Data/1_Scraped/Unstructured_Data/Global/coronavirus_news.csv')
    df_aapl= pd.read_csv('../Data/1_Scraped/Unstructured_Data/Stock/aapl_news.csv')
    df_meta= pd.read_csv('../Data/1_Scraped/Unstructured_Data/Stock/meta_news.csv')
    df_tsla= pd.read_csv('../Data/1_Scraped/Unstructured_Data/Stock/tsla_news.csv')

    # Removing duplicates
    df_world = df_world.drop_duplicates(subset=['Date', 'Title'], keep='first')
    df_politics = df_politics.drop_duplicates(subset=['Date', 'Title'], keep='first')
    df_coronavirus = df_coronavirus.drop_duplicates(subset=['Date', 'Title'], keep='first')
    df_aapl = df_aapl.drop_duplicates(subset=['Date', 'Title'], keep='first')
    df_meta = df_meta.drop_duplicates(subset=['Date', 'Title'], keep='first')
    df_tsla = df_tsla.drop_duplicates(subset=['Date', 'Title'], keep='first')
    
    # Further filtering based on keyword
    # aapl_keyword = ['Apple', 'iPhone', 'Macbook', 'iPad']
    # meta_keyword = ['Facebook', 'Meta', 'Metaverse']
    # tsla_keyword = ['Tesla', 'EV', 'Elon', 'Musk']

    # df_aapl = df_aapl[df_aapl['Title'].str.contains('|'.join(aapl_keyword)) == True]
    # df_meta = df_meta[df_meta['Title'].str.contains('|'.join(meta_keyword)) == True]
    # df_tsla = df_tsla[df_tsla['Title'].str.contains('|'.join(tsla_keyword)) == True]
    
    # Drop any Nan (Only present in these 2 dataframes)
    df_world = df_world.dropna()
    df_politics = df_politics.dropna()
    df_coronavirus = df_coronavirus.dropna()
    df_aapl = df_aapl.dropna()
    df_meta = df_meta.dropna()
    df_tsla = df_tsla.dropna()
    
    return df_world, df_politics, df_coronavirus, df_aapl, df_meta, df_tsla
    
def finbert_all():
    df_world, df_politics, df_coronavirus, df_aapl, df_meta, df_tsla = clean_all_data()
    
    df_aapl = finbert_process(df_aapl)
    df_meta = finbert_process(df_meta)
    df_tsla = finbert_process(df_tsla)
    # df_world = finbert_process(df_world)
    # df_politics = finbert_process(df_politics)
    # df_coronavirus = finbert_process(df_coronavirus)

    df_aapl.to_csv('../Data/2_Processed/Unstructured_Data/Stock/aapl_finbert.csv',index=False)
    df_meta.to_csv('../Data/2_Processed/Unstructured_Data/Stock/meta_finbert.csv',index=False)
    df_tsla.to_csv('../Data/2_Processed/Unstructured_Data/Stock/tsla_finbert.csv',index=False)
    # df_world.to_csv('../Data/2_Processed/Unstructured_Data/Global/world_finbert.csv',index=False)
    # df_politics.to_csv('../Data/2_Processed/Unstructured_Data/Global/politics_finbert.csv',index=False)
    # df_coronavirus.to_csv('../Data/2_Processed/Unstructured_Data/Global/coronavirus_finbert.csv',index=False)

def aggregate_sentiment_all(title, type='abs_max'):
    df_world_finbert= pd.read_csv('../Data/2_Processed/Unstructured_Data/Global/world_finbert.csv')
    df_politics_finbert= pd.read_csv('../Data/2_Processed/Unstructured_Data/Global/politics_finbert.csv')
    df_coronavirus_finbert= pd.read_csv('../Data/2_Processed/Unstructured_Data/Global/coronavirus_finbert.csv')
    df_aapl_finbert= pd.read_csv('../Data/2_Processed/Unstructured_Data/Stock/aapl_finbert.csv')
    df_meta_finbert= pd.read_csv('../Data/2_Processed/Unstructured_Data/Stock/meta_finbert.csv')
    df_tsla_finbert= pd.read_csv('../Data/2_Processed/Unstructured_Data/Stock/tsla_finbert.csv')

    aggregated_sentiment_aapl = sentiment_aggregator(df_aapl_finbert, title=title, type=type)
    aggregated_sentiment_meta = sentiment_aggregator(df_meta_finbert, title=title, type=type)
    aggregated_sentiment_tsla = sentiment_aggregator(df_tsla_finbert, title=title, type=type)
    # aggregated_sentiment_world = sentiment_aggregator(df_world_finbert, title=title, type=type)
    # aggregated_sentiment_politics = sentiment_aggregator(df_politics_finbert, title=title, type=type)
    # aggregated_sentiment_coronavirus = sentiment_aggregator(df_coronavirus_finbert, title=title, type=type)

    lst = [aggregated_sentiment_aapl, aggregated_sentiment_meta, aggregated_sentiment_tsla] #, aggregated_sentiment_world, aggregated_sentiment_politics, aggregated_sentiment_coronavirus]
    keys = ["AAPL", "META", "TSLA"] #, "World", "Politics", "Coronavirus"]
    
    return pd.concat(lst, keys=keys, axis=1)

In [104]:
finbert_all() #Takes quite long to run

Breaking data into smaller chunks
Processing chunk
Apple Extends Decline as BofA Downgrades to Neutral on More Balanced Risk/Reward
Take Advantage of Pullback and Buy Apple Stock - Analyst
Fast Company shuts website after hack sends 'obscene' Apple News notifications
Russia demands Apple explain VK removal from App Store
Wall Street bounces off lows as UK steps in to calm bonds
Wall Street ends sharply higher as Treasury yields dip
5 Rumors Circling Wall Street Today
Stock Market Today: Dow Racks Up Gains Amid Plunging Treasury Yields
S&P 500 Rides Slump in U.S. Treasury Yields Higher
Midday Movers: Biogen, Apple, DocuSign and More
Apple Production Reports Sound 'a Little Weird': TF International
Apple falls on fears of slowing iPhone 14 demand
U.S. Stocks Rise as Bond Yields Ease
Apple, Qualcomm, Lyft Fall Premarket; Biogen, Netflix Rise
Apple drops plan to boost iPhone production as demand falters-Bloomberg
Dow Futures Largely Flat; Fed Speakers, Apple Sales in Focus
Bond Rout Lifts 

In [106]:
df_all = aggregate_sentiment_all(title=True)
df_all = df_all.sort_values(by="Date")
df_all.to_csv('../Data/2_Processed/Unstructured_Data/All/all_finbert.csv')

1664